### TODO

- Correct it, it's finding local mimima
- Find the right way to turn the solution into integers

In [10]:
import numpy as np
import scipy.optimize as optimize
from random import randint
import sys

%xmode plain

from simulationmodel import *
from viz import *
%load_ext autoreload
%autoreload 2

Exception reporting mode: Plain
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
prob_death_transport = 0
num_days = 10
num_nodes = 4
node_capacities, transport_capacities, demands = init_simulation(num_nodes, 
                                                                 num_days, 
                                                                 icu_capacities=[30000, 20, 80, 1000],
                                                                 transport_capacities=[50, 60, 70, 80], 
                                                                 demand_min = 60, demand_max=100)
print(f"""Randomly generated
{num_days} days, {num_nodes} hospitals
ICU capacities of hospitals: {node_capacities}
Daily transport capacities: {transport_capacities}, 
Daily simulated demand (from uniform distrution): 
{demands}""")

Randomly generated
10 days, 4 hospitals
ICU capacities of hospitals: [30000, 20, 80, 1000]
Daily transport capacities: [50, 60, 70, 80], 
Daily simulated demand (from uniform distrution): 
[[96, 83, 73, 91, 79, 79, 99, 88, 74, 78], [67, 98, 81, 77, 76, 97, 92, 83, 67, 94], [76, 74, 63, 74, 68, 62, 82, 60, 87, 75], [71, 81, 86, 74, 88, 63, 68, 63, 71, 63]]


In [38]:
## ADD THIS

scenario_info = {
     'node_capacities' : node_capacities # how much capacity in each hospital. Size: [Nodes]
    ,'transport_capacities' : transport_capacities # Size: [Nodes]
    ,'demands' : demands #Size: [Days, Nodes]
    ,'num_days': num_days # int 
    ,'num_nodes':num_nodes # int
    ,'node_distances' : None #Size: [Nodes, Nodes]
    ,'prob_death_transport': 0 #float   
}

In [13]:
def split_n(num, num_nodes):
    #TODO this function is not truly random
    pieces = []
    for idx in range(num_nodes-1):
        pieces.append(randint(1,num-sum(pieces)-num_nodes+idx))

    pieces.append(num-sum(pieces))
    return pieces

In [14]:
## ADD scenario_info to f_cons

#define constraints
cons = []
for node in range(num_nodes):
    for day in range(num_days):
        cons.append({'type': 'ineq', 'fun': f_cons(node,day, scenario_info)})

        
#no movement matrix
no_movement = np.array([[[0]*num_nodes]*num_nodes]*num_days)

In [15]:
def random_init(random_days = False, movement_intensity = 1):
    #almost everyone moves everywhere
    random_movement = no_movement.copy()
    for day in range(num_days):
        for node_x in range(num_nodes):
            split_nodes = split_n(np.ceil(transport_capacities[node_x]*movement_intensity), num_nodes)
            for node_y in range(num_nodes):
                if node_x != node_y: #the ones that move to "self" are actually staying
                    random_movement[day, node_x, node_y] += split_nodes[node_y]
                    
    return random_movement

max_movement = random_init()
half_movement = random_init(movement_intensity=0.5)
one_ten_movement = random_init(movement_intensity=0.1)

In [16]:
one_ten_movement.ravel()

array([0, 1, 1, 2, 1, 0, 1, 2, 1, 3, 0, 2, 1, 3, 1, 0, 0, 1, 1, 2, 1, 0, 2,
       2, 1, 1, 0, 4, 2, 2, 2, 0, 0, 1, 1, 2, 2, 0, 1, 2, 2, 2, 0, 2, 3, 1,
       2, 0, 0, 1, 1, 2, 1, 0, 1, 3, 1, 2, 0, 3, 1, 3, 2, 0, 0, 1, 1, 2, 1,
       0, 1, 3, 2, 1, 0, 2, 2, 1, 2, 0, 0, 1, 1, 2, 1, 0, 1, 2, 3, 1, 0, 2,
       3, 2, 1, 0, 0, 1, 1, 2, 1, 0, 2, 2, 2, 2, 0, 2, 4, 1, 1, 0, 0, 1, 1,
       2, 1, 0, 1, 2, 1, 2, 0, 3, 3, 1, 1, 0, 0, 1, 1, 2, 2, 0, 1, 2, 2, 1,
       0, 2, 1, 4, 1, 0, 0, 1, 1, 2, 1, 0, 1, 2, 3, 1, 0, 2, 3, 1, 2, 0])

In [41]:
%%time

## ADD scenario_info as lambda to objective function
## ADD scenario_info to calc_total_deaths


#all the movements configuration and flatten them with ravel()
movements = [no_movement, 
             random_init(), 
             random_init(movement_intensity=0.75), 
             random_init(movement_intensity=0.5), 
             random_init(movement_intensity=0.25), 
             random_init(movement_intensity=0.1)]
#movements = [x.ravel() for x in movements]

#parameters to minimize
min_deaths = sys.maxsize
best_result = None
result_matrix = None
for movement_intensity in np.arange(0, 1.1, 0.1):
    print(f"Intensity:{movement_intensity:.1f}...", end="")
    movement = random_init(movement_intensity=movement_intensity) if movement_intensity else no_movement
    movement = movement.ravel()
    bounds_movement = generate_bounds(scenario_info)
    #the actual
    result = optimize.minimize(lambda x: calc_total_deaths(x, scenario_info), 
                               movement,
                               method='SLSQP', 
                               constraints=cons, 
                               bounds=bounds_movement)
    
    current_deaths = calc_total_deaths(np.rint(result.x), scenario_info)
    if min_deaths > current_deaths: 
        print("best! ", end="")
        min_deaths = int(current_deaths)
        best_result = result
        result_matrix = np.reshape(np.rint(result.x),(num_days,num_nodes,num_nodes))
print("Done")

Intensity:0.0...best! Intensity:0.1...best! Intensity:0.2...

KeyboardInterrupt: 

In [ ]:
scenario_info = {
     'node_capacities' : node_capacities # how much capacity in each hospital. Size: [Nodes]
    ,'transport_capacities' : transport_capacities # Size: [Nodes]
    ,'demands' : demands #Size: [Days, Nodes]
    ,'num_days': num_days # int 
    ,'num_nodes':num_nodes # int
    ,'node_distances' : None #Size: [Nodes, Nodes]
    ,'transport_cost': 0 #float   
}

In [ ]:
plot_scenario_overview(movements, [scenario_info for _ in range(len(movements))])

In [ ]:
len(movements)

In [ ]:
import matplotlib.pyplot as plt

def plot_deaths(deaths, ax=None):
    
    if type(deaths) != dict:
        print('wrong data type, expecting dictionary of arrays')
        print(type(deaths))
        return
    if ax is None:
        ax=plt.gca()
        
    ax.plot(deaths['unattended'].sum(axis=1), label='unattended', c='r')
    ax.plot(deaths['transport'].sum(axis=1), label='transport', c='g')
    ax.plot(deaths['total'].sum(axis=1), label='total', c='k')
#     ax2 = ax.twinx()
#     ax2.plot(deaths['total'].sum(axis=1).cumsum(), label='cumulativesum')
    
    ax.legend()

def plot_scenario_overview(movements, titles = None):
    """
        movement is a list of array[day, node_from, node_to]
    """
    if type(movements) != list:
        print('wrong data type, excepting a list of movements')
        return
    
    maxcols = 4
    rows = 1+len(movements)//maxcols
    cols = min(len(movements),maxcols)
    
    fig, subps = plt.subplots(rows,cols,sharey='all', figsize=(4*cols,4*rows))
    
    subps = subps.flatten()
    
    for i, m in enumerate(movements):
        ax = subps[i]
        if titles is None:
            ax.set_title(f'Transport Scenario {i}')
        else:
            ax.set_title(titles[i])
            
        plot_deaths(calc_daily_deaths(movements=m), ax=ax)

#     ax = subps[1]
#     ax.set_title('Transport Scenario 1')
#     plot_deaths(calc_daily_deaths(movements=movements[1]), ax=ax)
#     ax = subps[2]
#     ax.set_title('Transport Scenario 2')
#     plot_deaths(calc_daily_deaths(movements=movements[2]), ax=ax)

plot_scenario_overview(movements)

In [ ]:
num_days, num_nodes
node_capacities, transport_capacities, demands

In [ ]:
movements[0]

### These are the deaths that we get with inaction

In [ ]:
calc_total_deaths(no_movement)

### This is the best result found

In [ ]:
min_deaths

### This the movement matrix

In [ ]:
result_matrix

In [ ]:
for node in range(num_nodes):
    for day in range(num_days):
        print(f'Node: {node}, Day: {day}, Demand: {demand_day_node(np.rint(result.x),day,node)}')

In [ ]:
demand_day_node(np.rint(result.x),3,2)

# SANDBOX

In [ ]:
prob_death_transport = 0
num_days = 1
num_nodes = 2
node_capacities, transport_capacities, demands, movements_orig = init_simulation(num_nodes, num_days)
print(f"""Randomly generated
{num_days} days, {num_nodes} hospitals
ICU capacities of hospitals: {node_capacities}
Daily transport capacities: {transport_capacities}, 
Daily simulated demand (from uniform distrution): 
{demands}""")

movements = movements_orig.ravel()